In [1]:
import cvxpy as cp
import numpy as np

NameError: name 'ana' is not defined

In [2]:
cost = np.array([3]).reshape(-1,1)
demand = np.array([[1.5],[2],[3.5],[4],[5.5]]).reshape(-1,1)
produce = np.array([8])

In [3]:
def solve_second_stage_subproblem(produce,all_demand):

    selling_price = np.array([5]).reshape(-1,1)
    salvage_value = np.array([1]).reshape(-1,1)

    duals = np.array([])
    vs = np.array([])

    for demand in all_demand:

        sell = cp.Variable(shape=(1,1))
        salvage = cp.Variable(shape=(1,1))

        objective = cp.Minimize(-1*(selling_price.T@sell)-1*(salvage_value.T@salvage))
        constraints =  [sell+salvage<=produce,sell<=demand[0],salvage>=[0],sell>=[0]]

        problem = cp.Problem(objective, constraints)
        problem.solve()

        dual_value = problem.constraints[0].dual_value

        duals = np.append(duals,dual_value)
        vs = np.append(vs,problem.value)
    

    return duals,vs

In [4]:
duals,qs = solve_second_stage_subproblem(produce,demand)

In [5]:
def solve_first_stage_problem(produce,limit,g_ks,alpha_ks):
    
    cost = np.array([3]).reshape(-1,1)

    produce  = cp.Variable(shape=(1,1))
    v = cp.Variable(shape=(1,1))

    constraints = []
    for i in range(0,len(g_ks)):
        constraints.append(np.array([g_ks[i]]).T@produce+np.array([alpha_ks[i]])<=v)

    constraints.extend([produce>=0,v>=-1000000,produce>=limit,produce<=max(demand)])

    objective = cp.Minimize((cost.T@produce)+v)

    problem = cp.Problem(objective, constraints)
    problem.solve()

    print(problem.value)

    return produce.value, v.value

In [6]:
produce = np.array([3])
limit = -100000

g_ks = np.array([[]])
alpha_ks = np.array([[]])

for i in range(0,10):
    duals,qs = solve_second_stage_subproblem(produce,demand)

    senario = 5
    senario_prob = (1/senario)*(np.ones_like(demand))

    g_ks_temp = -senario_prob.T@duals
    alpha_ks_temp = senario_prob.T@qs - g_ks_temp.T@produce

    g_ks = np.append(g_ks,g_ks_temp)
    alpha_ks = np.append(alpha_ks,alpha_ks_temp)

    produce,limit= solve_first_stage_problem(produce,limit,g_ks,alpha_ks)
    print(produce)
    # print(limit)

    

-5.0000000008339605
[[5.49999999]]
-4.459767163612472
[[4.14941791]]
-4.299999996677485
[[3.74999999]]
-4.199999993635306
[[3.49999999]]
-4.199999989925718
[[3.49999998]]
-4.199999983394266
[[3.49999999]]
-4.199999985401114
[[3.49999999]]
-4.199999983646592
[[3.49999999]]
-4.199999985139872
[[3.49999997]]
-4.199999993809657
[[3.49999998]]
